In [1]:
import pandas as pd

temp_df = pd.read_csv('emails.csv')
temp_df = temp_df.truncate(after=10000)

In [2]:
print(temp_df)

                           file  \
0         allen-p/_sent_mail/1.   
1        allen-p/_sent_mail/10.   
2       allen-p/_sent_mail/100.   
3      allen-p/_sent_mail/1000.   
4      allen-p/_sent_mail/1001.   
5      allen-p/_sent_mail/1002.   
6      allen-p/_sent_mail/1003.   
7      allen-p/_sent_mail/1004.   
8       allen-p/_sent_mail/101.   
9       allen-p/_sent_mail/102.   
10      allen-p/_sent_mail/103.   
11      allen-p/_sent_mail/104.   
12      allen-p/_sent_mail/105.   
13      allen-p/_sent_mail/106.   
14      allen-p/_sent_mail/107.   
15      allen-p/_sent_mail/108.   
16      allen-p/_sent_mail/109.   
17       allen-p/_sent_mail/11.   
18      allen-p/_sent_mail/110.   
19      allen-p/_sent_mail/111.   
20      allen-p/_sent_mail/112.   
21      allen-p/_sent_mail/113.   
22      allen-p/_sent_mail/114.   
23      allen-p/_sent_mail/115.   
24      allen-p/_sent_mail/116.   
25      allen-p/_sent_mail/117.   
26      allen-p/_sent_mail/118.   
27      allen-p/_sen

In [3]:
import string
import time

#Two ways to make sure not to include parts with a : but not part of body
#1 make a list with all the first bits I want to ignore
exclusion_data = ['Message-ID', 'Date', 'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Cc', 'Bcc']
#2 while in the loop, create a new list by .split(":") and adding list.[0] to words not to include, but probably long, but benefit of updating?
#Solutions above aren't perfect, probably a better one to improve on later

#Takes string and array of column data to keep
def extract_data(message, compulsary_data):
    email = {}
    lines = message.split('\n')
    lines = list(filter(None, lines))
    filled_data = []
    body = ''
    #Go over every line
    for line in lines:
        #If the line was already added move onto the next line.
        added = False
        #Go and check for every required data
        for data in compulsary_data:
            #If the line was already used skip checking and move onto the next line
            if added:
                break
            #If the data to check was already filled then ignore it - consider just removing data from compulsary_data
            if data in filled_data:
                continue
            #If : is in the line
            if ":" in line:
                #Is either non-body data to include
                if line.startswith((data + ":")):
                    split_line = line.split(':')
                    email[data] = split_line[1].lower()
                    filled_data.append(data)
                    added = True
                #Or is body data or data to be excluded
                else:
                    #If it is part of the data to be excluded, ignore
                    ignore_line = False
                    for exclusion in exclusion_data:
                        if line.startswith((exclusion + ":")):
                            ignore_line = True
                            break
                    #If it isn't meant to be excluded then it's in the body.
                    if not ignore_line:
                        body += line
                        added = True
            else:
                body += line
                added = True
    #Fix this later. Forwarded emails also have data but are ignored.
    email['Body'] = body
    if body is not "":
        filled_data.append('Body')
    #Final check, if any of the required data is missing, return nothing.
    for key in compulsary_data:
        if key not in filled_data:
            return None
    return email
    
def generate_df():
    compulsary_data = ['From', 'To', 'Body']
    columns = {}
    for message in temp_df['message']:
        email = extract_data(message, compulsary_data)
        if email is None:
            continue
        else:
            for key, value in email.items():
                columns.setdefault(key, []).append(value)
    return columns
   # return df

columns = generate_df()
    
#Get my columns of name, body, to, from
#Get my tfidf from it
#DBSCAN that

#tfidf by converting into matrix

#df = generate_df()


#idf = vectorizer.idf_
#print dict(zip(vectorizer.get_feature_names(), idf))

#Turn into bag of words, 3 for from, to, body.

In [4]:
print(len(columns['Body']))
print(len(columns['To']))
print(len(columns['From']))

9689
9689
9689


In [5]:
df = pd.DataFrame(columns)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Body'])
#X = StandardScaler().fit_transform(X)


In [7]:
from sklearn.cluster import DBSCAN
from sklearn import metrics
db = DBSCAN(eps=0.3, min_samples=10).fit(X)

In [8]:
import numpy as np

core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
    
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    print(xy[:5])
    print(X[:5])
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

  (0, 24901)	0.012565561236801581
  (0, 43893)	0.016745449907742155
  (0, 14345)	0.023037155274876115
  (0, 48061)	0.014677448546218552
  (0, 22798)	0.014828010488208292
  (0, 10311)	0.01979170951911645
  (0, 29442)	0.013460452337657145
  (0, 14486)	0.023037155274876115
  (0, 28307)	0.01821443668573619
  (0, 53552)	0.023037155274876115
  (0, 45717)	0.016603373539353802
  (0, 46341)	0.023037155274876115
  (0, 17123)	0.021058589115536146
  (0, 42461)	0.022225476787918984
  (0, 41239)	0.015499312581001378
  (0, 54646)	0.017427890818038753
  (0, 41498)	0.023037155274876115
  (0, 23507)	0.020609519053177806
  (0, 12770)	0.016093655368956645
  (0, 34303)	0.023037155274876115
  (0, 31992)	0.03652417205631056
  (0, 51120)	0.01934813568117707
  (0, 28878)	0.018310497792327338
  (0, 11017)	0.023037155274876115
  (0, 41496)	0.04043713380353718
  :	:
  (4, 37297)	0.05973314000327673
  (4, 54513)	0.01630507278323225
  (4, 19604)	0.01283002253933308
  (4, 35898)	0.01733309185462471
  (4, 12216)	0.10

ValueError: setting an array element with a sequence.

In [ ]:
#More things to do:
#tfidf
#standardize?
#lemmatize
#stem
#stopwords
#noisy labels are assigned -1, so make sure to differentiate them from the other colors
# Now we print the top terms across all documents.

#Y are labels

def top_feats_per_cluster(X, y, features, min_tfidf=0.1, top_n=25):
    dfs = []

    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label) 
        feats_df = top_mean_feats(X, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
return dfs

X = vect.fit_transform(email_df.body)
features = vect.get_feature_names()

print(top_mean_feats(X, features, None, 0.1, 10))

def top_mean_feats(X, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    if grp_ids:
        D = X[grp_ids].toarray()
    else:
        D = X.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    
return top_tfidf_feats(tfidf_means, features, top_n)


In [ ]:
print(__doc__)

import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)

print('Estimated number of clusters: %d' % n_clusters_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:

    dbscan = cluster.DBSCAN(eps=params['eps'])
       for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # catch warnings related to kneighbors_graph
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # add black color for outliers (if any)
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()